# Mesh Generation in 3D using GMSH

## Import Packages

In [2]:
import Gmsh: gmsh 
using GR 
using LinearAlgebra
using Plots
using LaTeXStrings

In [3]:
gmsh.initialize()

## Section 1:/ Mesh on Disk  

In [4]:
r = 2; # radius of circle 
mshd  = r / 5; # Mesh density at inner boundary

gmsh.model.add("coax_cable")
geo = gmsh.model.geo;

## Points
geo.addPoint(0, 0, 0, mshd/10, 1)
geo.addPoint(r, 0, 0, mshd/10, 2)
geo.addPoint(-r, 0, 0, mshd, 3)

## Curves
geo.addCircleArc(2, 1, 3, 1)
geo.addCircleArc(3, 1, 2, 2)

## Surfaces
geo.addCurveLoop([1, 2], 1)
geo.addPlaneSurface([1], 1)

# Generate mesh
geo.synchronize();
gmsh.model.mesh.generate(2)

#..if true, write mesh to file for further processing 
if (false) gmsh.write("disk-2D.msh") end 
#..if true, visualize mesh through the GUI 
if (false) gmsh.fltk.run() end 

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 50%] Meshing curve 2 (Circle)
Info    : Done meshing 1D (Wall 0.00503279s, CPU 0.005001s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0114595s, CPU 0.011373s)
Info    : 591 nodes 1181 elements


## Section 2:/ First (Failed) Attempt to Extrude Mesh of Disk to Mesh on Cylinder 

In [6]:
?gmsh.model.geo.extrude

```
gmsh.model.geo.extrude(dimTags, dx, dy, dz, numElements = Cint[], heights = Cdouble[], recombine = false)
```

Extrude the entities `dimTags` in the built-in CAD representation, using a translation along (`dx`, `dy`, `dz`). Return extruded entities in `outDimTags`. If `numElements` is not empty, also extrude the mesh: the entries in `numElements` give the number of elements in each layer. If `height` is not empty, it provides the (cumulative) height of the different layers, normalized to 1. If `recombine` is set, recombine the mesh in the layers.

Return `outDimTags`.


In [ ]:
gmsh.clear()
gmsh.model.add("coax_cylinder")
geo = gmsh.model.geo;

## Points
geo.addPoint(0, 0, 0, mshd/10, 1)
geo.addPoint(r, 0, 0, mshd/10, 2)
geo.addPoint(-r, 0, 0, mshd, 3)

## Curves
geo.addCircleArc(2, 1, 3, 1)
geo.addCircleArc(3, 1, 2, 2)

## Surfaces
geo.addCurveLoop([1, 2], 1)
geo.addPlaneSurface([1], 1)

ov = gmsh.model.geo.extrude([(2,1)], 0, 0, 2)

# Generate mesh
geo.synchronize();
gmsh.model.mesh.generate(3)

#..if true, write mesh to file for further processing 
if (false) gmsh.write("t1.msh") end 
#..if true, visualize mesh through the GUI 
if (true) gmsh.fltk.run() end 

## Section 3:/ 3D Mesh on Cuboidal Domain from Tutorial t3 (here used as reference)

In [15]:
# Clear all models and create a new one
gmsh.clear()
gmsh.model.add("t3")

# Copied from `t1.jl'...
lc = 0.5
gmsh.model.geo.addPoint(0, 0, 0, lc, 1)
gmsh.model.geo.addPoint(1, 0,  0, lc, 2)
gmsh.model.geo.addPoint(1, 1, 0, lc, 3)
gmsh.model.geo.addPoint(0, 1, 0, lc, 4)
gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(3, 2, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 1, 4)
gmsh.model.geo.addCurveLoop([4, 1, -2, 3], 1)
gmsh.model.geo.addPlaneSurface([1], 1)
gmsh.model.geo.synchronize()
gmsh.model.addPhysicalGroup(1, [1, 2, 4], 5)
gmsh.model.addPhysicalGroup(2, [1], -1, "My surface")

# As in `t2.jl', we plan to perform an extrusion along the z axis.  But
# here, instead of only extruding the geometry, we also want to extrude the
# 2D mesh. This is done with the same `extrude()' function, but by
# specifying element 'Layers' (2 layers in this case, the first one with 8
# subdivisions and the second one with 2 subdivisions, both with a height of
# h/2). The number of elements for each layer and the (end) height of each
# layer are specified in two vectors:
h = 1

ov = gmsh.model.geo.extrude([(2, 1)], 0, 0, h)

# gmsh.model.addPhysicalGroup(3, [ov[1][1]], 101)

geo.synchronize();
gmsh.model.mesh.generate(3)
gmsh.write("t3.msh")
if (false) gmsh.fltk.run() end 

Info    : Clearing all models and views...
Info    : Done clearing all models and views
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 8 (Line)
Info    : [ 50%] Meshing curve 9 (Line)
Info    : [ 50%] Meshing curve 10 (Line)
Info    : [ 60%] Meshing curve 11 (Line)
Info    : [ 70%] Meshing curve 13 (Line)
Info    : [ 80%] Meshing curve 14 (Line)
Info    : [ 90%] Meshing curve 18 (Line)
Info    : [100%] Meshing curve 22 (Line)
Info    : Done meshing 1D (Wall 0.001753s, CPU 0.001659s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 15 (Surface, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 19 (Surface, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 23 (Surface, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 27 (Surface, Frontal-Delaunay)
Inf

In [16]:
    # Clear all models and create a new one
    gmsh.clear()
    gmsh.model.add("t3")

    # Copied from `t1.jl'...
    lc = 1e-2
    gmsh.model.geo.addPoint(0, 0, 0, lc, 1)
    gmsh.model.geo.addPoint(.1, 0,  0, lc, 2)
    gmsh.model.geo.addPoint(.1, .3, 0, lc, 3)
    gmsh.model.geo.addPoint(0, .3, 0, lc, 4)
    gmsh.model.geo.addLine(1, 2, 1)
    gmsh.model.geo.addLine(3, 2, 2)
    gmsh.model.geo.addLine(3, 4, 3)
    gmsh.model.geo.addLine(4, 1, 4)
    gmsh.model.geo.addCurveLoop([4, 1, -2, 3], 1)
    gmsh.model.geo.addPlaneSurface([1], 1)
    gmsh.model.geo.synchronize()
    gmsh.model.addPhysicalGroup(1, [1, 2, 4], 5)
    gmsh.model.addPhysicalGroup(2, [1], -1, "My surface")

    # As in `t2.jl', we plan to perform an extrusion along the z axis.  But
    # here, instead of only extruding the geometry, we also want to extrude the
    # 2D mesh. This is done with the same `extrude()' function, but by
    # specifying element 'Layers' (2 layers in this case, the first one with 8
    # subdivisions and the second one with 2 subdivisions, both with a height of
    # h/2). The number of elements for each layer and the (end) height of each
    # layer are specified in two vectors:
    h = 0.1
    angle = 90.

    ov = gmsh.model.geo.extrude([(2, 1)], 0, 0, h, [8, 2], [0.5, 1])

    # The extrusion can also be performed with a rotation instead of a
    # translation, and the resulting mesh can be recombined into prisms (we use
    # only one layer here, with 7 subdivisions). All rotations are specified by
    # an an axis point (-0.1, 0, 0.1), an axis direction (0, 1, 0), and a
    # rotation angle (-Pi/2):
    ov = gmsh.model.geo.revolve([(2, 28)], -0.1, 0, 0.1, 0, 1, 0, -pi / 2, [7])

    # Using the built-in geometry kernel, only rotations with angles < Pi are
    # supported. To do a full turn, you will thus need to apply at least 3
    # rotations. The OpenCASCADE geometry kernel does not have this limitation.

    # A translation (-2 * h, 0, 0) and a rotation ((0, 0.15, 0.25), (1, 0, 0),
    # angle * Pi / 180) can also be combined to form a "twist".  The last
    # (optional) argument for the extrude() and twist() functions specifies
    # whether the extruded mesh should be recombined or not. The `angle'
    # parameter is retrieved from the ONELAB database (it can be set
    # interactively in the GUI -- see below):
    angle = gmsh.onelab.getNumber("Parameters/Twisting angle")[1]
    ov = gmsh.model.geo.twist([(2, 50)], 0, 0.15, 0.25, -2 * h, 0, 0, 1, 0, 0,
                              angle * pi / 180., [10], [], true)

    gmsh.model.geo.synchronize()

    # All the extrusion functions return a vector of extruded entities: the
    # "top" of the extruded surface (in `ov[1]'), the newly created volume (in
    # `ov[2]') and the tags of the lateral surfaces (in `ov[3]', `ov[4]', ...).

    # We can then define a new physical volume (with tag 101) to group all the
    # elementary volumes:
    gmsh.model.addPhysicalGroup(3, [1, 2, ov[2][2]], 101)

    gmsh.model.mesh.generate(3)
    gmsh.write("t3.msh")

Info    : Clearing all models and views...
Info    : Done clearing all models and views


LoadError: BoundsError: attempt to access 0-element Vector{Float64} at index [1]